In [318]:
import pandas as pd
import altair as alt
import geopandas as gpd
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Недоліки попередньо даного графіку:
1. Незручно читати області / регіони (велике нагромадження)
2. Для порівняння двох років - дуже багато всього на одному графіку, можна було би зробити розбиття інакше
3. ВДВ - це who? 
4. В заголовку незрозуміло, про які регіони йдеться 
5. Сітка (grid) складно сприймається візуально, так як дуже маленька і часта
6. Бар чарти не посортовані за шкалою % відносно областей чи відносно складових
7. Складно порівнювати між собою сруктуру і її складові між різними областями, через те, що немає єдиного відліку, для того щоб відштовхуватисью Легко порівняти лише нижню і верхню частину бар чарту, так як вони мають 'спільний' початок
8. Легенда займає половину графіку

In [319]:
data = pd.read_excel('1.4_02.xlsx')
data.head()

,"Регіон, рік",сільське господарство,промисловість,будівництво,послуги,державне управління
0,"Україна, 2010",8.3,25.9,3.3,57.3,5.2
1,"Україна, 2017",12.1,25.2,2.5,53.7,6.5
2,"Вінницька, 2010",21.6,19.3,4.1,47.4,7.6
3,"Вінницька, 2017",28.1,21.4,2.0,39.8,8.7
4,"Волинська, 2010",16.9,15.6,3.2,57.7,6.6


In [320]:
data['сільське господарство'] = data['сільське господарство'] / 100
data['промисловість'] = data['промисловість'] / 100
data['будівництво'] = data['будівництво'] / 100
data['послуги'] = data['послуги'] / 100
data['державне управління'] = data['державне управління'] / 100

In [321]:
data['Регіон'] = [d.split(',')[0] for d in data['Регіон, рік']]
data['Рік'] = [int(d.split(',')[1]) for d in data['Регіон, рік']]
data.head()

,"Регіон, рік",сільське господарство,промисловість,будівництво,послуги,державне управління,Регіон,Рік
0,"Україна, 2010",0.083,0.259,0.033,0.573,0.052,Україна,2010
1,"Україна, 2017",0.121,0.252,0.025,0.537,0.065,Україна,2017
2,"Вінницька, 2010",0.216,0.193,0.041,0.474,0.076,Вінницька,2010
3,"Вінницька, 2017",0.281,0.214,0.020,0.398,0.087,Вінницька,2017
4,"Волинська, 2010",0.169,0.156,0.032,0.577,0.066,Волинська,2010


## Vis 1

Для того, щоб зробити графік більш зрозумілим, ми маємо подумати, яке розбиття на підграфіки було би найзручніше робити. Як ми знаємо, ми маємо 5 різних категорій із структури господарства, та 20+ різних областей. Також маємо два роки (2010б 2017). Основні цілі, яким ми маємо слідувати:
- зобразити області, структуру (ділення чогось цілого (100%) на частини), рік (2010, 2017), та %.
- Отримати змогу добре порівняти 2010 і 2017 роки та їх структуру області А
- Отримати змогу добре порівняти 2010 роки та їх структуру області А та Б між собою

Початкові ідеї:
- плиткова карта з чартами на кожну область (pie chart / bar chart)
    нагромадження не зміниться, буде все ще важко порівняти дві області між собою
- перенести назви областей на іншу вісь 
- додати інтерактив для кращого розуміння при виділенні певних об'єктів
- змінити заголовок 

Для того, щоб імплементувати ідеї та зробити графік зрозумілішим, я спершу взяла за основу - розбиття графіків на підграфіки за одиницею структури. 

In [322]:
lines = alt.Chart(data).mark_line().encode(
    x = alt.X('Регіон:N', title=None, 
#               scale=alt.Scale(domain=[1,len(data)+1], nice=False),
#               axis=alt.Axis(tickCount=data.shape[0], 
#             labels=False,
#             values=[i for i in range(1, len(data)+1)], grid=False),
              sort=alt.Sort(field='сільське господарство', op='max', order='descending')),
    
    y = alt.Y('сільське господарство:Q', 
              scale=alt.Scale(domain=[0,1], nice=False), 
              axis=alt.Axis(grid=True, format='%')),
              
    color=alt.Color('Регіон:N', legend=None, scale=alt.Scale(range=['grey'])))

points = alt.Chart(data).mark_circle(size = 100).encode(
    x = alt.X('Регіон:N', sort=alt.Sort(field='сільське господарство', op='max', order='descending')),
    y = alt.Y('сільське господарство:Q'),
    color  = alt.Color('Рік:Q', scale=alt.Scale(range=['pink', 'purple']), legend=None),
    tooltip = alt.Tooltip('Рік:Q')
)

alt.layer(lines, points).properties(width = 800, height = 600)

alt.LayerChart(...)

In [323]:
data.columns

Index(['Регіон, рік', 'сільське господарство', 'промисловість', 'будівництво',
       'послуги', 'державне управління', 'Регіон', 'Рік'],
      dtype='object')

In [324]:
data.columns = ['Region, year', 'Cільське господарство', 'Промисловість', 'Будівництво', 
                'Послуги', 'Державне управління','Region', 'Year']

In [325]:
cols_reference = {'Agriculture':'Cільське господарство', 'Industry':'Промисловість', 'Construction':'Будівництво',
       'Services':'Послуги', 'Governance':'Державне управління'
}

In [326]:
color = []
for y in data['Year']:
    if y == 2010:
        color.append('pink')
    else:
        color.append('purple')

data['color'] = color
data.head()

,"Region, year",Cільське господарство,Промисловість,Будівництво,Послуги,Державне управління,Region,Year,color
0,"Україна, 2010",0.083,0.259,0.033,0.573,0.052,Україна,2010,pink
1,"Україна, 2017",0.121,0.252,0.025,0.537,0.065,Україна,2017,purple
2,"Вінницька, 2010",0.216,0.193,0.041,0.474,0.076,Вінницька,2010,pink
3,"Вінницька, 2017",0.281,0.214,0.020,0.398,0.087,Вінницька,2017,purple
4,"Волинська, 2010",0.169,0.156,0.032,0.577,0.066,Волинська,2010,pink


In [327]:
select_region = alt.selection_single(on = 'mouseover', 
                                     fields = ['Region'], 
                                     nearest = True, empty = 'all')

In [328]:
input_dropdown = alt.binding_select(options = data.Region.unique())
select_region_dropdown = alt.selection_single(fields = ['Region'], bind = input_dropdown, 
                                     name = 'Select', empty = 'all')

У цьому (та майже у всіх інших) я додала селектор за областю, для того, щоб мати змогу виділити певну область і мати змогу порівняти структуру ВДВ за роками. Два кольори позначають роки, що дає візуально сприйняття і розуміння, якого року показник тієї чи іншої сфери був вищий, а де нижчий. Єдине - не подобається те, що трохи важко орієнтуватися в такого типу графіках і візуально сприймати теж важко (особисто мені), тому я вирішила спробувати щось інше також.

In [329]:
lines = alt.Chart(data).mark_line().transform_fold(
   list(data.columns[1:6]),
   as_ = ["column", "value"]
).encode(
      x = alt.X('Region:N', title=None, 
              sort=alt.EncodingSortField('value', op='max', order='descending')),
    
    y = alt.Y('max(value):Q', 
              scale=alt.Scale(domain=[0,1], nice=False), 
              axis=alt.Axis(grid=True, format='%', title='')),
    y2 = alt.Y2('min(value):Q'),
    strokeWidth = alt.value(3),
    color = alt.condition(select_region_dropdown, 
                          alt.ColorValue('#615756'), alt.ColorValue('#615756')),
    tooltip = [alt.Tooltip('Region:N'), alt.Tooltip('max(value):Q', format='.2%'), 
              alt.Tooltip('min(value):Q', format='.2%')],
    
    )

# lines
points = alt.Chart(data).transform_fold(
   list(data.columns[1:6]),
   as_ = ["column", "value"]
).mark_circle(size = 100).encode(
    x = alt.X('Region:N', sort=alt.EncodingSortField('value', op='max', order='descending')),
    y = alt.Y('value:Q'),
    color  = alt.Color('Year:N', scale=alt.Scale(range=['#242F40', '#CCA43B']),
                      legend=alt.Legend(title='Рік')),
    tooltip = [alt.Tooltip('Region:N'), alt.Tooltip('Year:N'), alt.Tooltip('value:Q', format='.2%')]
)

# points
alt.layer(lines, points, data=data).transform_fold(
   list(data.columns[1:6]),
   as_ = ["column", "value"]
).add_selection(select_region_dropdown).encode(opacity=alt.condition(
        select_region_dropdown,
        alt.value(0.8),
        alt.value(0.01)
    )).properties(
    width = 260, height = 500).facet(facet=alt.Facet('column:N',
                        sort=alt.EncodingSortField('value', 
                                op='max', order='descending'), 
title='Зміна структури ВДВ у розрізі областей України за 2017 рік порівняно з 2010 роком',
    header=alt.Header(titleFontSize=20, labelFontSize=12))).resolve_scale(
    x='independent')                                   
#                     op='max', order='descending')),columns=3).resolve_scale(x='independent')

alt.FacetChart(...)

## Vis 2

Тут я просто поміняла місцями вісі - виглядає жахливо - цей варіант точно НІ.

In [330]:
lines = alt.Chart(data).transform_fold(
   list(data.columns[1:6]),
   as_ = ["column", "value"]
).mark_line().encode(
    x = alt.X('max(value):Q', scale=alt.Scale(domain=[0,1], nice=False), 
               axis=alt.Axis(grid=True, format='%', title='')),
    x2 = alt.X2('min(value):Q'),
    y = alt.Y('Region:N', sort=alt.Sort(field='value', op='max', order='descending'),
             axis=alt.Axis(title='')),
    strokeWidth = alt.value(3),
    color = alt.condition(select_region_dropdown, alt.ColorValue('#615756'), alt.ColorValue('#615756')),
    tooltip = [alt.Tooltip('Region:N'), alt.Tooltip('max(value):Q', format='.2%'), 
              alt.Tooltip('min(value):Q', format='.2%')],)

points = alt.Chart(data).transform_fold(
   list(data.columns[1:6]),
   as_ = ["column", "value"]
).mark_circle(size = 100).encode(
    x = alt.X('value:Q'),
    y = alt.Y('Region:N', sort=alt.Sort(field='value', op='max', order='descending')),
    color  = alt.Color('Year:N', scale=alt.Scale(range=['#242F40', '#CCA43B']),
                      legend=alt.Legend(title='Рік')),
)

alt.layer(lines, points, data=data).transform_fold(
   list(data.columns[1:6]),
   as_ = ["column", "value"]
).properties(
    width = 1500/5, height = 500).add_selection(select_region_dropdown).encode(opacity=alt.condition(
        select_region_dropdown,
        alt.value(0.8),
        alt.value(0.01)
    )).facet(
    facet=alt.Facet('column:N',
                        sort=alt.EncodingSortField('value', 
                                op='max', order='descending'), 
title='Зміна структури ВДВ у розрізі областей України за 2017 рік порівняно з 2010 роком',
    header=alt.Header(titleFontSize=20, labelFontSize=12))).resolve_scale(x='independent')

alt.FacetChart(...)

## Vis 3

In [331]:
grid = pd.read_csv('grid.csv')
grid.head()

,row,col,code,name
0,1,2,1,Volyn
1,1,5,2,Kyiv City
2,1,6,3,Chernihiv
3,1,7,4,Sumy
4,2,1,5,L'viv


In [332]:
match_cities = {'Івано-Франківська': "Ivano-Frankivs'k",
 'Волинська':'Volyn', 
 'Вінницька':'Vinnytsya',
 'Дніпропетровська': "Dnipropetrovs'k",
 'Донецька':"Donets'k",
 'Житомирська':'Zhytomyr',
 'Закарпатська':'Transcarpathia',
 'Запорізька':'Zaporizhzhya',
 'Київська':'Kyiv',
 'Кіровоградська':'Kirovohrad',
 'Луганська':"Luhans'k",
 'Львівська':"L'viv",
 'Миколаївська':'Mykolayiv',
 'Одеська':'Odesa',
 'Полтавська':'Poltava',
 'Рівненська':'Rivne',
 'Сумська':'Sumy',
 'Тернопільська':"Ternopil'",
 'Харківська':'Kharkiv',
 'Херсонська':'Kherson',
 'Хмельницька':"Khmel'nyts'kyy",
 'Черкаська':'Cherkasy',
 'Чернівецька':'Chernivtsi',
 'Чернігівська':'Chernihiv',
 'м.Київ':'Kyiv City'}
# set(list(data['Регіон']))

In [333]:
match_cities_rev = {v:k for k, v in match_cities.items()}

In [334]:
grid = grid[grid['name'] != "Sevastopol'"]
grid = grid[grid['name'] != "Crimea"]

In [335]:
new_data_country = []
data_new = data.copy()
for c in grid['name']:
    new_data_country.append(match_cities_rev.get(c, 'None'))

grid['name'] = new_data_country
grid.head()

,row,col,code,name
0,1,2,1,Волинська
1,1,5,2,м.Київ
2,1,6,3,Чернігівська
3,1,7,4,Сумська
4,2,1,5,Львівська


In [337]:
data_new = data_new[data_new['Region'] != 'Україна']
data_new.head()

,"Region, year",Cільське господарство,Промисловість,Будівництво,Послуги,Державне управління,Region,Year,color
2,"Вінницька, 2010",0.216,0.193,0.041,0.474,0.076,Вінницька,2010,pink
3,"Вінницька, 2017",0.281,0.214,0.020,0.398,0.087,Вінницька,2017,purple
4,"Волинська, 2010",0.169,0.156,0.032,0.577,0.066,Волинська,2010,pink
5,"Волинська, 2017",0.172,0.154,0.021,0.577,0.076,Волинська,2017,purple
6,"Дніпропетровська, 2010",0.047,0.477,0.018,0.431,0.027,Дніпропетровська,2010,pink


In [338]:
select_region = alt.selection_single(on = 'mouseover', 
                                     fields = ['column'], 
                                     nearest = False, empty = 'all')

In [339]:
base = alt.Chart(data_new).transform_fold(
   list(data_new.columns[1:6]),
   as_ = ["column", "value"]
).mark_line(point = True).encode(
    x = alt.X(field = 'Year', type = 'nominal', axis = alt.Axis(labelAngle=0, title='')),
    y = alt.Y(field = 'value', type = 'quantitative', 
              scale = alt.Scale(domain = [0, 1]),
              axis = alt.Axis(format='%', title='')),
    color = alt.Color('column:N', legend=alt.Legend(title='')),
    detail = alt.Detail('Region:N'),
    tooltip = alt.Tooltip(['Region:N', 
                           'Year:Q', 'value:Q']),
 opacity=alt.condition(
        select_region,
        alt.value(0.8),
        alt.value(0.1)
    ),
).add_selection(select_region).properties(
    width = 120, height = 120)
base

alt.Chart(...)

In [340]:
title = alt.Chart(data_new).mark_text(dy = -70).encode(
    text = alt.Text('Region:N', aggregate='max')
)

base += title
base

alt.LayerChart(...)

In [341]:
empty = alt.Chart(alt.Data(values = [{'x': 1990, 'y':-10}, {'x': 2020, 'y': -10}])).mark_point(opacity = 0).encode(
    x = alt.X('x:Q', scale = alt.Scale(domain = [1988, 2020], nice = False), 
              axis=alt.Axis(title=None, values=[1990, 2020], 
                            format = 'f', labelOpacity = 0, domainOpacity = 0, tickOpacity=0, grid=False)),
    y = alt.Y('y:Q', axis=alt.Axis(title=None, labelOpacity = 0, domainOpacity = 0, tickOpacity=0, grid=False))
).properties(width = 120, height = 120)

empty

alt.Chart(...)

In [342]:
chart = alt.vconcat()
for row in grid.row.unique():
  chart_row = alt.hconcat()
  for col in range(1, 9):
    if col in grid[grid.row == row].col.unique():
      chart_row |= base.transform_filter(alt.datum.Region == grid[(grid.row == row) & (grid.col == col)].name.values.tolist()[0])
    else:
      chart_row |= empty
  chart &= chart_row


Тут, власне, ми все ж створили плиткову карту з областями, де кожна область має свій підграфік, який показує структуру ВДВ на 2010 та 2017 роки.

Цей варіант мені подобається, але важко порівнювати регіони між собою. Також, я додала інтерактив для виділення лише окремих частин структури ВДВ на кожну область.

In [343]:
chart.add_selection(select_region).properties(
title = alt.TitleParams(
    text = 'Зміна структури ВДВ у розрізі областей України за 2017 рік порівняно з 2010 роком',
    fontSize=22, dx=300, dy=-40)
  ).configure_view(strokeWidth = 0)

alt.VConcatChart(...)

## Vis 4

Остання візуалізація, яка на мою думку, є найкращою, наведена нижче. Тут можна було би ще додати +1 інтерактив для вибору року, який хочеш дивитись, я намагалась, але не вийшло :(

In [344]:
data_2010 = data[data['Year']==2010]
data_2017 = data[data['Year']==2017]
data_general = data_2010.merge(data_2017, left_on='Region', right_on='Region')
data_general.head()

,"Region, year_x",Cільське господарство_x,Промисловість_x,Будівництво_x,Послуги_x,Державне управління_x,Region,Year_x,color_x,"Region, year_y",Cільське господарство_y,Промисловість_y,Будівництво_y,Послуги_y,Державне управління_y,Year_y,color_y
0,"Україна, 2010",0.083,0.259,0.033,0.573,0.052,Україна,2010,pink,"Україна, 2017",0.121,0.252,0.025,0.537,0.065,2017,purple
1,"Вінницька, 2010",0.216,0.193,0.041,0.474,0.076,Вінницька,2010,pink,"Вінницька, 2017",0.281,0.214,0.020,0.398,0.087,2017,purple
2,"Волинська, 2010",0.169,0.156,0.032,0.577,0.066,Волинська,2010,pink,"Волинська, 2017",0.172,0.154,0.021,0.577,0.076,2017,purple
3,"Дніпропетровська, 2010",0.047,0.477,0.018,0.431,0.027,Дніпропетровська,2010,pink,"Дніпропетровська, 2017",0.073,0.482,0.018,0.384,0.043,2017,purple
4,"Донецька, 2010",0.038,0.458,0.033,0.444,0.027,Донецька,2010,pink,"Донецька, 2017",0.072,0.499,0.017,0.350,0.062,2017,purple


In [373]:
select_legend = alt.selection_single(on = 'click', 
                                     fields = ['Year_x', 'Year_y'], bind = 'legend', nearest = False, empty = 'all')

In [374]:
input_dropdown = alt.binding_select(options = data_general.Region.unique())
select_region_dropdown = alt.selection_single(fields = ['Region'], bind = input_dropdown, 
                                     name = 'Select', empty = 'all')

In [379]:
chart = []
for col_name in data.columns[1:6]:
    
    bars_2010 = alt.Chart(data_general).mark_bar(color='red').encode(
          x = alt.X('Region:N', title=None, 
                  sort=alt.EncodingSortField(f'{col_name}_y', op='max', order='descending')),

        y = alt.Y(f'{col_name}_x:Q',
                  scale=alt.Scale(domain=[0,1], nice=False), 
                  axis=alt.Axis(grid=True, format='%')),
        tooltip = [alt.Tooltip('Region:N', title='Регіон'), 
                   alt.Tooltip(f'{col_name}_x:N', title=f'{col_name} в 2010', format='.2%'),
                    alt.Tooltip(f'{col_name}_y:N', title=f'{col_name} в 2017', format='.2%')],
                   opacity = alt.condition(
            select_region_dropdown | select_legend,
            alt.value(0.7),
            alt.value(0.05)),
        color=alt.Color('Year_x:N', legend=alt.Legend(title='Рік'))
        )


    bars_2017 = alt.Chart(data_general).mark_bar(color='blue').encode(
          x = alt.X('Region:N', title=None, 
                  sort=alt.EncodingSortField(f'{col_name}_y', op='max', order='descending')),

        y = alt.Y(f'{col_name}_y:Q', 
                  title=None,
                  scale=alt.Scale(domain=[0,1], nice=False), 
                  axis=alt.Axis(grid=True, format='%')),
        tooltip = [alt.Tooltip('Region:N', title='Регіон'), 
                   alt.Tooltip('Year_y:N', title='Рік'), 
                   alt.Tooltip(f'{col_name}_y:N', title=col_name, format='.2%')],
            opacity = alt.condition(
            select_region_dropdown | select_legend,
            alt.value(0.7),
            alt.value(0.05)),
            color=alt.Color('Year_y:N', legend=alt.Legend(title='Рік'))
                   )

    
    # bars_2017
    layer = alt.layer(bars_2010, bars_2017, data=data_general).encode(
                tooltip = [alt.Tooltip('Region:N', title='Регіон'), 
                   alt.Tooltip('Year_y:N', title='Рік'), 
                   alt.Tooltip(f'{col_name}_y:N', 
                               title=col_name, 
                               format='.2%')]).add_selection(select_region_dropdown, select_legend).properties(
        width = 260, height = 500, title=col_name)
    
    chart.append(layer)

In [380]:
three_concat=alt.hconcat(chart[3].properties(width = 250, height =300), 
            chart[1].properties(width = 250, height = 300),
            chart[0].properties(width = 250, height = 300)
           )

two_concat = alt.hconcat(chart[4].properties(width = 250, height = 300),
            chart[2].properties(width = 250, height = 300))

alt.hconcat(three_concat, two_concat, data=data_general).add_selection(select_region_dropdown, 
                                                                       select_legend).properties(
    background = '#F9F9F9', padding = 25, title = alt.TitleParams(
    text = 'Зміна структури ВДВ у розрізі областей України за 2017 рік порівняно з 2010 роком',
    fontSize=22, dx=280, dy=-40)
  ).configure_view(
    strokeWidth=0)

alt.HConcatChart(...)

За допомогою цієї візуалізації ми можемо легко порівнювати два роки за структурою для одного регіону використовуючи селектор, але при цьому можемо подивитися на розподіл кожної сфери, порівняти дві області між собою, бо всі частини мають однакову початкову точку.

Я також поміняла заголовок, щоб було зрозуміло про які регіони йдеться.